# Step 1: Set up the environment

In [1]:
import os
import google.generativeai as genai

if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.")
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")


Running on COLAB environment.


In [2]:
# Clone the data repository into colab
!git clone https://github.com/openknowledge/workshop-genai-data.git
PROCESSED_DATA_PATH = "/content/workshop-genai-data/processed/gutenberg/"
EVALUATION_DATA_PATH = "/content/workshop-genai-data/evaluation/"

Cloning into 'workshop-genai-data'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 28 (delta 6), reused 20 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 436.74 KiB | 4.50 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [3]:
# import colab specific lib to read user data (aka colab managed secrets)
from google.colab import userdata

In [4]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model.
# Note: GEMINI_API_KEY must be set as COLAB userdata before!
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
# Install additional libraries
%%capture
!pip install -qU langchain-text-splitters
!pip install chromadb

In [6]:
# Import additional libraries
from langchain_text_splitters import RecursiveCharacterTextSplitter
from chromadb import EphemeralClient
import requests
import re
import uuid
import json
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import pandas as pd
from pathlib import Path
import time
import numpy as np


In [7]:
# Configure pandas display options
pd.set_option("max_colwidth", None)

In [8]:
# Set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-1.5-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.0
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT = " "

# Set defaults for retrieval
DEFAULT_K = 3
DEFAULT_CHUNK_OVERLAP = 100
DEFAULT_CHUNK_SIZE = 2000

## Define helper functions

In [9]:
# This will be the chromadb collection we use as a knowledge base. We do not need the in-memory EphemeralClient.
chromadb_client = EphemeralClient()
chromadb_collection = chromadb_client.create_collection(name="default")


In [10]:
def call_genai_model_for_completion(
        model_name: str = DEFAULT_MODEL,
        config_temperature:float = DEFAULT_CONFIG_TEMPERATURE,
        config_top_k: int = DEFAULT_CONFIG_TOP_K,
        config_max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS,
        system_prompt : str = DEFAULT_SYSTEM_PROMPT,
        user_prompt : str = DEFAULT_USER_PROMPT,
        verbose: bool = False
        ):

    if verbose:
        # print out summary of input values / parameters
        print(f'Generating answer for following config:')
        print(f'  - SYSTEM PROMPT used:\n {system_prompt}')
        print(f'  - USER PROMPT used:\n {user_prompt}')
        print(f'  - MODEL used:\n {model_name} (temperature = {config_temperature}, top_k = {config_top_k}, max_output_tokens = {config_max_output_tokens})')

    # create generation config
    model_config = genai.GenerationConfig(
        max_output_tokens=config_max_output_tokens,
        temperature=config_temperature,
        top_k=config_top_k
    )

    # create genai model with generation config
    genai_model = genai.GenerativeModel(
        model_name= model_name,
        generation_config= model_config
    )

    # Attention: We manipulated the safety settings
    response = genai_model.generate_content(
        contents=[system_prompt, user_prompt], safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    })
    return response

In [11]:
# RAG building blocks

# Get content of books. The content will already be cleansed.
def load_file_content(file_name: str) -> str:
  with open(f"{PROCESSED_DATA_PATH}{file_name}", "r") as f:
    return f.read()

# Building Block "Chunking": Split the content into smaller chunks
def do_chunk(text: str, chunk_size: int = DEFAULT_CHUNK_SIZE, chunk_overlap: int = DEFAULT_CHUNK_OVERLAP) -> list[str]:
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len,
  )
  return text_splitter.split_text(text=text)

# Building Block "Embedding": Create multi dimensional embeddings for a given chunk.
def do_embed(chunk: str) -> list[float]:
  return genai.embed_content(model=EMBEDDING_MODEL, content=chunk).get("embedding")

def do_batch_embed(chunks: list[str]) -> list[list[float]]:
  return genai.embed_content(model=EMBEDDING_MODEL, content=chunks).get("embedding")

# Building Block "Knowledgebase": Store embeddings and the corresponding content in a vectorstore
def persist_embeddings(chunks: list[str], embeddings: list[float], clear_knowledgebase: bool)-> None:
  if clear_knowledgebase:
    chromadb_collection = chromadb_client.delete_collection(name="default")

  chromadb_collection = chromadb_client.get_or_create_collection(name="default")
  # Persist the embeddings and the chunks in the knowledgebase
  ids = [str(uuid.uuid4()) for _ in chunks]
  chromadb_collection.add(ids=ids, documents=chunks, embeddings=embeddings)

# Building Block "Augmentation": Create an updated prompt by merging the original user input with the provided context
# Attention: We manipulated the augmented prompt in order to see the guardrails in action
def augment(user_input: str, context: list[str]) -> str:
  prepared_context = "\n".join(context)
  augmented_prompt = f"""
    Answer the question as detailed as possible from the provided context. If the answer is not in
    provided context just say, 'answer is not available in the context', don't provide the wrong answer.
    Respond short and concisely.
    Context:\n{prepared_context}?\n
    Question: \n{user_input}\n

    Answer:
  """
  return augmented_prompt

# Building Block "Top-k Fetching": Get the k semantically closest chunks to the user input from the knowledgebase
def do_top_k_fetching(user_input_embedding: list[float], k: int) -> tuple[list[str],list[float]]:
  # Since we will do the fetching always only for one user_input,
  # instead of querying for multiple embeddings simultanously as allowed by the choma API,
  # we add the embeddings below to a list and return only the first document (chunk)
  chromadb_collection = chromadb_client.get_or_create_collection(name="default")
  results = chromadb_collection.query(query_embeddings=[user_input_embedding], n_results=k)
  return (results["documents"][0], results["distances"][0]) # Return the distances to get better insights

# Building Block "Generation": Use the generation model to create a response
def generate_response(prompt: str) -> str:
  completion_result = call_genai_model_for_completion(
      model_name=GENERATION_MODEL,
      user_prompt=prompt,
  )
  return completion_result.text

In [12]:
def do_ingestion(file_names: list[str], chunk_size: int = DEFAULT_CHUNK_SIZE, clear_knowledgebase: bool = False, verbose: bool = True) -> None:
  # Ingest file by file
  for file_name in file_names:
    # Load prepared book content
    file_content = load_file_content(file_name)

    # Chunk the content into smaller chunks
    chunks = do_chunk(file_content, chunk_size=chunk_size)
    if verbose:
      print(f'Loaded {len(chunks)} chunks from {file_name}')

    # Embed the chunks
    embeddings = do_batch_embed(chunks)

    # Persist the embeddings and the chunks in the knowledgebase
    persist_embeddings(chunks, embeddings, clear_knowledgebase)

In [13]:
def do_rag(user_input: str, k: int = DEFAULT_K, verbose: bool = False) -> tuple[str, list[str], list[float]]:
  # Embed the user input
  user_input_embedding = do_embed(chunk=user_input)

  # "R" like "Retrieval": Get the k semantically closest chunks to the user input from the knowledgebase
  retrieval_start_time = time.time()
  (context, distances) = do_top_k_fetching(user_input_embedding=user_input_embedding, k=k)
  if verbose:
    retrieval_end_time = time.time()
    retrieval_time = round(retrieval_end_time - retrieval_start_time, 2)

  # "A" like "Augmented": Create the augmented prompt
  augmented_prompt = augment(user_input=user_input, context=context)

  # "G" like "Generation": Generate a response
  generation_start_time = time.time()
  response = generate_response(prompt=augmented_prompt)
  if verbose:
    generation_end_time = time.time()
    generation_time = round(generation_end_time - generation_start_time, 2)
    print(f'Retrieval took {retrieval_time}s. Generation took {generation_time}s.')

  return (response, context, distances)


In [14]:
# Define a custom exception
class FactCheckingValidationError(Exception):
  pass

# Define a response format
class FactCheckingValidationAnswer(typing.TypedDict):
    is_grounded: bool


def guard_fact_checking(bot_response: str, context: list[str]) -> str:
  # Prepare the context to be used in the guard prompt
  context = "\n".join(context)

  # Define the prompt for the guardrail
  guard_prompt = f"""
    You are given a task to identify if the answer is grounded and entailed to the context.
    You will only use the contents of the context and not rely on external knowledge.
    'context': {context} 'answer': {bot_response}
    """

  # Call the guardrail model with the desired output format
  model = genai.GenerativeModel(GUARDING_MODEL)
  result = model.generate_content(
      guard_prompt,
      generation_config=genai.GenerationConfig(
          response_mime_type="application/json", response_schema=FactCheckingValidationAnswer
      ),
  )

  # Evaluate the validation
  fact_checking_validation = json.loads(result.text)
  if not fact_checking_validation["is_grounded"]:
    error_msg = f"The bot answer '{bot_response}' is not grounded in the context '{context}'"
    raise FactCheckingValidationError(error_msg)
  return bot_response

In [15]:
def do_fact_checked_rag(user_input: str,k: int = DEFAULT_K, verbose: bool = False) -> tuple[str, list[str], list[float]]:
    (answer, context, distances) = do_rag(user_input=user_input, k=k, verbose=verbose)
    guarded_output = guard_fact_checking(bot_response=answer, context=context)
    return (guarded_output, context, distances)

In [46]:
def print_insights(dataframe: pd.DataFrame):
  median_response_time = round(dataframe['response_time'].median(), 2)
  median_min_distance = round(dataframe['min_context_distance'].median(), 2)
  median_median_distance = round(dataframe['mean_context_distance'].median(), 2)
  n_hallucinations = dataframe['min_context_distance'].isna().sum()

  print(f'Number of detected hallucinations: {n_hallucinations}')
  print(f'Median response time: {median_response_time} seconds')
  print(f'Median of minimum distance: {median_min_distance}')
  print(f'Median of median distance: {median_median_distance}')

# Step 2: Configure the genAI models

In [17]:
# GenAI Models
GENERATION_MODEL = "gemini-1.5-flash"
EMBEDDING_MODEL = "models/text-embedding-004"
GUARDING_MODEL = "gemini-1.5-flash-8b"

# Step 3: Define initial values for parameters to be adjusted

In [18]:
# Ingesting
KNOWLEDGEBASE_CONTENT = set(['study_in_scarlett.txt'])
CHUNK_SIZE = 2000

# Retrieval
TOP_K = 3

# Step 4: Prepare the knowledgebase

In [19]:
do_ingestion(file_names=KNOWLEDGEBASE_CONTENT)

Loaded 126 chunks from study_in_scarlett.txt


# Step 5: Manually evaluate responses

In [20]:
# Read csv from local files
evaluation_dataframe = pd.read_csv(EVALUATION_DATA_PATH + 'simple_evaluation_dataset.csv')
evaluation_dataframe.head()

,story_name,question,ground_truth_answer
0,A Study in Scarlet,What year does Dr. Watson complete his medical degree?,1878
1,A Study in Scarlet,Where do Sherlock Holmes and Dr. Watson decide to live together?,221B Baker Street
2,A Study in Scarlet,What word does Sherlock Holmes find written in blood on the wall at the crime scene?,RACHE
3,A Study in Scarlet,What profession does Sherlock Holmes describe himself as having?,A consulting detective
4,A Study in Scarlet,What clue suggests to Holmes that the murderer might have smoked a particular kind of cigar?,Ashes from a Trichinopoly cigar found at the crime scene


In [50]:
# The function gives us the desired outputs to gather some insights
def generate_rag_answers(dataframe: pd.DataFrame, k: int = TOP_K, verbose: bool = False):
  def generate_rag_response_with_insights(question: str) -> pd.Series:
        # Define start time for calculating the response time
        start_time = time.time()

        # Generate the RAG response
        try:
            output = do_fact_checked_rag(question, k=k, verbose=verbose)
            response = output[0]  # Extract the response from the output
            distances = output[2]  # Extract the distances from the output
            min_distance = round(min(distances), 2)  # Find the minimum distance
            median_distance = np.nanmedian(distances)  # Calculate the mean distance
        except FactCheckingValidationError:
            # Return a predefined response if the fact checking validation fails
            response = "Possible hallucination detected."
            min_distance = None
            median_distance = None

        # Define end time for calculating the response time
        end_time = time.time()

        # Calculate response time
        response_time = round(end_time - start_time, 2)

        # Return response and insights
        return pd.Series([response, response_time, min_distance, median_distance])

  # Apply the insights function to each row and store results in new columns
  dataframe[['rag_response', 'response_time', 'min_context_distance', 'median_context_distance']] = dataframe['question'].apply(generate_rag_response_with_insights)
  return dataframe


In [22]:
# Generate responses and insights
evaluation_dataframe = generate_rag_answers(evaluation_dataframe)
evaluation_dataframe

,story_name,question,ground_truth_answer,rag_response,response_time,min_context_distance,mean_context_distance
0,A Study in Scarlet,What year does Dr. Watson complete his medical degree?,1878,1878 \n,2.47,0.83,0.95
1,A Study in Scarlet,Where do Sherlock Holmes and Dr. Watson decide to live together?,221B Baker Street,They decide to live together at 221B Baker Street. \n,2.04,0.84,0.86
2,A Study in Scarlet,What word does Sherlock Holmes find written in blood on the wall at the crime scene?,RACHE,RACHE \n,1.91,0.63,0.69
3,A Study in Scarlet,What profession does Sherlock Holmes describe himself as having?,A consulting detective,Sherlock Holmes describes himself as a **consulting detective**. \n,1.99,0.64,0.73
4,A Study in Scarlet,What clue suggests to Holmes that the murderer might have smoked a particular kind of cigar?,Ashes from a Trichinopoly cigar found at the crime scene,"Holmes found scattered ash on the floor that was dark in color and flakey, which is characteristic of a Trichinopoly cigar. \n",2.23,0.69,0.75
5,A Scandal in Bohemia,Who is the woman that Sherlock Holmes admires and refers to as 'the woman'?,Irene Adler,Possible hallucination detected.,2.06,NaN,NaN
6,The Red-Headed League,What unique characteristic qualified Mr. Jabez Wilson to join the Red-Headed League?,His bright red hair,Possible hallucination detected.,2.06,NaN,NaN
7,A Case of Identity,Who is Miss Mary Sutherland engaged to marry?,Hosmer Angel,Possible hallucination detected.,2.03,NaN,NaN
8,The Boscombe Valley Mystery,Who is initially suspected of killing Charles McCarthy?,"His son, James McCarthy",Possible hallucination detected.,1.93,NaN,NaN
9,The Five Orange Pips,Who is ultimately responsible for the threats and deaths in the Openshaw family?,The Ku Klux Klan,Possible hallucination detected.,2.02,NaN,NaN


In [23]:
print_insights(evaluation_dataframe)

Number of detected hallucinations: 10
Mean response time: 2.08 seconds
Mean minimum distance: 0.73
Mean mean distance: 0.8


# Step 6: Ingest remaining stories

In [24]:
# Ingest the second book, which contains the other stories
KNOWLEDGEBASE_CONTENT.add('adventures_of_sherlock_holmes.txt')
do_ingestion(file_names=KNOWLEDGEBASE_CONTENT, clear_knowledgebase=False)

Loaded 126 chunks from study_in_scarlett.txt
Loaded 304 chunks from adventures_of_sherlock_holmes.txt


In [27]:
# Generate responses and insights
evaluation_dataframe = generate_rag_answers(evaluation_dataframe)
evaluation_dataframe

,story_name,question,ground_truth_answer,rag_response,response_time,min_context_distance,mean_context_distance
0,A Study in Scarlet,What year does Dr. Watson complete his medical degree?,1878,1878 \n,3.05,0.83,0.89
1,A Study in Scarlet,Where do Sherlock Holmes and Dr. Watson decide to live together?,221B Baker Street,They decide to live together at 221B Baker Street. \n,2.01,0.83,0.84
2,A Study in Scarlet,What word does Sherlock Holmes find written in blood on the wall at the crime scene?,RACHE,RACHE \n,2.19,0.63,0.65
3,A Study in Scarlet,What profession does Sherlock Holmes describe himself as having?,A consulting detective,"Sherlock Holmes describes himself as a ""consulting detective"". \n",1.94,0.64,0.68
4,A Study in Scarlet,What clue suggests to Holmes that the murderer might have smoked a particular kind of cigar?,Ashes from a Trichinopoly cigar found at the crime scene,"Holmes found the ash of an Indian cigar, which he identified based on his knowledge of tobacco ashes. \n",1.93,0.59,0.66
5,A Scandal in Bohemia,Who is the woman that Sherlock Holmes admires and refers to as 'the woman'?,Irene Adler,Irene Adler \n,3.20,0.78,0.78
6,The Red-Headed League,What unique characteristic qualified Mr. Jabez Wilson to join the Red-Headed League?,His bright red hair,Mr. Jabez Wilson's red hair qualified him to join the Red-Headed League. \n,2.20,0.55,0.62
7,A Case of Identity,Who is Miss Mary Sutherland engaged to marry?,Hosmer Angel,Miss Mary Sutherland is engaged to marry Mr. Hosmer Angel. \n,1.92,0.82,0.85
8,The Boscombe Valley Mystery,Who is initially suspected of killing Charles McCarthy?,"His son, James McCarthy","James McCarthy, the son of the deceased, is initially suspected of killing Charles McCarthy. \n",2.25,0.87,0.90
9,The Five Orange Pips,Who is ultimately responsible for the threats and deaths in the Openshaw family?,The Ku Klux Klan,Possible hallucination detected.,1.97,NaN,NaN


In [28]:
print_insights(evaluation_dataframe)

Number of detected hallucinations: 5
Mean response time: 2.18 seconds
Mean minimum distance: 0.73
Mean mean distance: 0.76


# Step 7: Manually adjust *chunk size*

In [61]:
# Define new chunk size. Be careful: The smaller the chunk_size, the more time is needed for the ingestion. I.e. adjusted_chunk_size = 400 takes 1-2 minutes
adjusted_chunk_size = 1000

In [62]:
# We need to clear the knowledgebase for this
do_ingestion(file_names=KNOWLEDGEBASE_CONTENT, chunk_size=adjusted_chunk_size, clear_knowledgebase=True, verbose=True)

Loaded 266 chunks from study_in_scarlett.txt
Loaded 640 chunks from adventures_of_sherlock_holmes.txt


In [63]:
# Generate responses and insights
evaluation_dataframe = generate_rag_answers(evaluation_dataframe, verbose=True)
evaluation_dataframe

Retrieval took 0.01s. Generation took 0.61s.
Retrieval took 0.01s. Generation took 0.64s.
Retrieval took 0.01s. Generation took 0.56s.
Retrieval took 0.01s. Generation took 0.61s.
Retrieval took 0.01s. Generation took 0.71s.
Retrieval took 0.01s. Generation took 0.66s.
Retrieval took 0.01s. Generation took 0.69s.
Retrieval took 0.01s. Generation took 0.64s.
Retrieval took 0.01s. Generation took 0.7s.
Retrieval took 0.01s. Generation took 0.67s.
Retrieval took 0.01s. Generation took 0.66s.
Retrieval took 0.01s. Generation took 0.79s.
Retrieval took 0.01s. Generation took 0.69s.
Retrieval took 0.01s. Generation took 0.67s.
Retrieval took 0.01s. Generation took 0.59s.


,story_name,question,ground_truth_answer,rag_response,response_time,min_context_distance,mean_context_distance,median_context_distance
0,A Study in Scarlet,What year does Dr. Watson complete his medical degree?,1878,Possible hallucination detected.,2.31,NaN,NaN,NaN
1,A Study in Scarlet,Where do Sherlock Holmes and Dr. Watson decide to live together?,221B Baker Street,Possible hallucination detected.,1.94,NaN,NaN,NaN
2,A Study in Scarlet,What word does Sherlock Holmes find written in blood on the wall at the crime scene?,RACHE,Possible hallucination detected.,1.97,NaN,NaN,NaN
3,A Study in Scarlet,What profession does Sherlock Holmes describe himself as having?,A consulting detective,Possible hallucination detected.,1.81,NaN,NaN,NaN
4,A Study in Scarlet,What clue suggests to Holmes that the murderer might have smoked a particular kind of cigar?,Ashes from a Trichinopoly cigar found at the crime scene,Holmes found the ash of an Indian cigar and identified it based on his knowledge of tobacco ashes. \n,1.88,0.67,0.73,0.687363
5,A Scandal in Bohemia,Who is the woman that Sherlock Holmes admires and refers to as 'the woman'?,Irene Adler,Possible hallucination detected.,1.91,NaN,0.78,NaN
6,The Red-Headed League,What unique characteristic qualified Mr. Jabez Wilson to join the Red-Headed League?,His bright red hair,Possible hallucination detected.,2.06,NaN,0.70,NaN
7,A Case of Identity,Who is Miss Mary Sutherland engaged to marry?,Hosmer Angel,Miss Mary Sutherland is engaged to marry Hosmer Angel. \n,1.91,0.83,NaN,0.876010
8,The Boscombe Valley Mystery,Who is initially suspected of killing Charles McCarthy?,"His son, James McCarthy","The initial suspect is Charles McCarthy's son, James McCarthy. \n",2.10,0.93,0.96,0.943639
9,The Five Orange Pips,Who is ultimately responsible for the threats and deaths in the Openshaw family?,The Ku Klux Klan,Possible hallucination detected.,1.89,NaN,0.94,NaN


In [64]:
print_insights(evaluation_dataframe)

Number of detected hallucinations: 9
Median response time: 1.91 seconds
Median of minimum distance: 0.75
Median of median distance: 0.78


# Step 8: Manually adjust *k*
We do not need to ingest again, since our adjustment just affects the retrieval pipeline. Keep in mind, that we use the chunking from before, because we did not update the knowledgebase.


In [33]:
adjusted_k = 30

In [ ]:
# Generate responses and insights
evaluation_dataframe = generate_rag_answers(evaluation_dataframe, k=adjusted_k)
evaluation_dataframe

In [ ]:
print_insights(evaluation_dataframe)

# Step 9: Define Learnings
- If we ingest more files, the hallucination will be reduced
- If we increase the *chunk_size*, distances go up. The context becomes more irrelevant
- If we increase *k* the response time increases. This effect is even more obvious, if the *chunk_size* is large. Even if we can reduce some hallucinations due to the increased context, the context quickly becomes more irrelevant as demonstrated by higher mean distances.
- This procedure needs to be automatized by using optimization techniques in addition to rag metrics.
- There are many more parameters to be adjusted like prompts, generation/embedding/guardrail models, chunk_overlap, types of splitters, types of retrievers, distance functions, distance thresholds, model parameters, [...]